<a href="https://colab.research.google.com/github/anespart1/educationuse/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC_%EA%B0%80%EC%A0%B8%EC%99%80_%EB%B0%80%EB%8F%84%EC%A7%80%EB%8F%84(Heatmap)_%EA%B7%B8%EB%A6%AC%EA%B8%B0_20210901.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터를 가져와 Folium으로 밀도지도(Heatmap) 그리기

##Biz-GIS Geocoding 서비스를 이용해 무료로 주소 좌표 변환하기(설명)
본 코드는 (주)Biz-GIS의 지오코딩 툴 프로그램을 사용해 좌표생성하는 것을 전제로 작성되었습니다.<br>
추후 Biz-GIS 프로그램의 저작권 또는 관련 서비스 조정(現 하루 1만개 제한) 등으로 인해 내용이 변경될 수 있음을 알려드립니다.

###1. Biz-GIS Geocoding 툴 검색
<br>
<img src="https://raw.githubusercontent.com/anespart1/educationuse/main/bizgis_geocoding1.JPG" width='800' /><br>


###2. Biz-GIS Geocoding 툴 다운로드
컴퓨터 환경에 맞는 비트 프로그램 다운로드
<br>
<img src="https://raw.githubusercontent.com/anespart1/educationuse/main/bizgis_geocoding2.JPG" width='800' /><br>

###3. Biz-GIS Geocoding 툴 압축풀기
<br>
<img src="https://raw.githubusercontent.com/anespart1/educationuse/main/bizgis_geocoding3.jpg" width='800' /><br>

###4. Biz-GIS Geocoding 툴 실행
GeocodingTool64.exe ->  [파일로 처리] 클릭 -> 엑셀(또는 CSV)파일 열기
<br>
<img src="https://raw.githubusercontent.com/anespart1/educationuse/main/bizgis_geocoding4.JPG" width='1000' /><br>

###5. Biz-GIS Geocoding 툴 환경설정
문자열 칼럼 구분자 선택, 주소 열 선택 등
<br>
<img src="https://raw.githubusercontent.com/anespart1/educationuse/main/bizgis_geocoding5.JPG" width='1000' /><br>

###6. Biz-GIS Geocoding 툴 지오코딩 실시
지오코딩 완료 후 [shp 만들기] 클릭
<br>
<img src="https://raw.githubusercontent.com/anespart1/educationuse/main/bizgis_geocoding6.JPG" width='800' /><br>

###7. Biz-GIS Geocoding 툴 dbf 파일 확인
[shp 만들기] 클릭 후 엑셀파일 위치에 댜앙한 파일이 생성된 것을 확인 가능<br>
이후 *.dbf 파일만 업로드해서 사용
<br>
<img src="https://raw.githubusercontent.com/anespart1/educationuse/main/bizgis_geocoding7.JPG" width='1000' /><br>

## 사용된 코드(숨김)

In [ ]:
import folium
from folium.plugins import *
import urllib
#print(folium.__version__)
!pip install simpledbf
from simpledbf import Dbf5
import numpy as np

In [ ]:
#dbf 파일 변환기
def dbf_to_pandas_df(dbf_url=None, lat_row=None, lon_row=None):
  if 'http' in dbf_url:
    link = urllib.parse.quote(dbf_url, safe=':/?-=') # 변경하지 않을 문자들
    urllib.request.urlretrieve(link, url.split('/')[-1])
    dbf = Dbf5(url.split('/')[-1], codec='euc-kr')
  else:
    dbf = Dbf5(url, codec='euc-kr')
  df = dbf.to_dataframe()
  #좌표값 숫자로 변환
  df[lon_row] = df[lon_row].apply(lambda x: float(x))
  df[lat_row] = df[lat_row].apply(lambda x: float(x))
  return df

In [ ]:
def spatial_visualizing(xcoord=None, ycoord=None, zoom= 7,
                        minimap=True, measure=True, position=True, fullscrean=True):
  m = folium.Map(location=[np.mean(ycoord), np.mean(xcoord)], zoom_start=zoom,
                 control_scale=True) #스케일바 추가
  #지도 타일 종류 추가
  for i in ['Stamen Terrain', 'Stamen Water Color', 'cartodbpositron']:
    folium.TileLayer(i).add_to(m)
  #미니맵 추가
  if minimap==True:
    m.add_child(MiniMap())
  #거리측량기 추가
  if measure==True:
    m.add_child(MeasureControl())
  #좌표기 추가
  if position==True:
    MousePosition(prefix='경위도 좌표:').add_to(m)
  #전체화면모드 추가
  if fullscrean==True:
    Fullscreen(title='전체화면', title_cancel='돌아가기', force_separate_button=True).add_to(m)

  return m

##전국 소방서 밀도지도(Heatmap) 그리기(Ctrl+Enter)
(119 안전센터는 포함되지 않습니다)

In [ ]:
#@title 가져올 DBF 파일 경로를 입력해주세요
url =  'https://raw.githubusercontent.com/anespart1/educationuse/master/소방청_시도_소방서_현황_20200121.dbf' #@param {type:"string"}
#@markdown (URL이 아닌 코랩에 업로드한 파일 경로도 가능합니다)
#@markdown ###위도(Y)값 열 이름을 입력해주세요 
lat='Y' #@param {type:"string"}
#@markdown ###경도(X)값 열 이름을 입력해주세요
lon='X' #@param {type:"string"}
df = dbf_to_pandas_df(dbf_url=url, lat_row=lat, lon_row=lon)
m = spatial_visualizing(xcoord=df[lon].mean(), ycoord=df[lat].mean(), zoom= 7)

#마커 클러스터 및 클러스터 그룹 추가
marker_cluster = folium.plugins.MarkerCluster(control=False)
m.add_child(marker_cluster)
group = folium.plugins.FeatureGroupSubGroup(marker_cluster, '소방서')
m.add_child(group)

#전국 소방서 추가
for n, station in df.iterrows():
  popup = folium.Popup('위도: {}<br>경도: {}'.format(station['Y'], station['X']),
                       max_width=300) #max_width를 설정하면 세로로 길게 나오는 현상을 피할 수 있음
  folium.Marker(location=[station['Y'], station['X']], popup=popup, tooltip=station['소방서'],
                icon=folium.Icon(color="red", icon="fire-extinguisher", prefix='fa')).add_to(group)
#Heatmap 추가
heatmap = folium.plugins.HeatMap(zip(df['Y'].values.tolist(), df['X'].values.tolist()),
                                 name='소방서 밀도', radius=22, min_opacity=0.28)
m.add_child(heatmap)
#레이어 컨트롤러 추가
folium.LayerControl().add_to(m)
m

In [ ]:
#지도 저장하기
m.save('heatmap.html')
from google.colab import files
files.download('/content/heatmap.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>